# Text vector search

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [2]:
APPLY = True
COLLECTION_NAME = '<var:table_name>' if not APPLY else 'sample_text_vector_search'

In [3]:
from superduper import superduper

db = superduper('mongomock:///test_db')

2025-Jan-13 12:45:15.81| INFO     | Duncans-MBP.fritz.box| superduper.misc.plugins:13   | Loading plugin: mongodb
2025-Jan-13 12:45:15.87| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:64   | Building Data Layer
2025-Jan-13 12:45:15.87| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:79   | Data Layer built
2025-Jan-13 12:45:15.87| INFO     | Duncans-MBP.fritz.box| superduper.backends.base.cluster:99   | Cluster initialized in 0.00 seconds.
2025-Jan-13 12:45:15.87| INFO     | Duncans-MBP.fritz.box| superduper.base.build:184  | Configuration: 
 +---------------+----------------------+
| Configuration |        Value         |
+---------------+----------------------+
|  Data Backend | mongomock:///test_db |
+---------------+----------------------+


<!-- TABS -->
## Get useful sample data

In [4]:
import json
import requests
import io

def getter():
    response = requests.get('https://superduperdb-public-demo.s3.amazonaws.com/text.json')
    return json.loads(response.content.decode('utf-8'))

In [5]:
if APPLY:
    data = getter()

<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [6]:
if APPLY:
    from superduper import Document
    ids = db.execute(db[COLLECTION_NAME].insert([Document({'x': x}) for x in data]))

2025-Jan-13 12:45:21.08| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (table, sample_text_vector_search) not found in cache, loading from db
2025-Jan-13 12:45:21.08| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:599  | Load (('table', 'sample_text_vector_search')) from metadata...
2025-Jan-13 12:45:21.08| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:331  | Table sample_text_vector_search does not exist, auto creating...
2025-Jan-13 12:45:21.57| WARNING  | Duncans-MBP.fritz.box| superduper.misc.annotations:296  | `superduper.ext.torch` is deprecated and will be removed in a future release. Please insteall `superduper_torch` and use `from superduper_torch import *` instead.
2025-Jan-13 12:45:21.59| WARNING  | Duncans-MBP.fritz.box| superduper.misc.annotations:296  | `superduper.ext.pillow` is deprecated and will be removed in a future release. Please insteall `superduper_pillow` and use `from superduper_pillow import *` instead

## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [7]:
from superduper import Model

class Chunker(Model):
    chunk_size: int = 200
    signature: str = 'singleton'

    def predict(self, text):
        text = text.split()
        chunks = [' '.join(text[i:i + self.chunk_size]) for i in range(0, len(text), self.chunk_size)]
        return chunks

Now we apply this chunker to the data by wrapping the chunker in `Listener`:

In [8]:
from superduper import Listener

upstream_listener = Listener(
    model=Chunker('chunk_model', chunk_size=200, example='test ' * 50),
    select=db[COLLECTION_NAME].select(),
    key='x',
    identifier=f'chunker_{COLLECTION_NAME}',
    flatten=True,
)

In [9]:
if APPLY:
    db.apply(upstream_listener, force=True)

2025-Jan-13 12:45:27.83| WARNING  | Duncans-MBP.fritz.box| superduper.components.listener:76   | output_table not found in listener.dict()
2025-Jan-13 12:45:27.84| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (model, chunk_model) not found in cache, loading from db
2025-Jan-13 12:45:27.84| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:599  | Load (('model', 'chunk_model')) from metadata...
2025-Jan-13 12:45:27.84| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:359  | Found new model:chunk_model:76436da6a29d4031
2025-Jan-13 12:45:27.85| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:593  | Component (listener, chunker_sample_text_vector_search) not found in cache, loading from db
2025-Jan-13 12:45:27.85| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:599  | Load (('listener', 'chunker_sample_text_vector_search')) from metadata...
2025-Jan-13 12:45:27.85| INFO     | Duncans-MBP.fritz.box| superduper.base.dat

187it [00:00, 29373.64it/s]

2025-Jan-13 12:45:27.88| INFO     | Duncans-MBP.fritz.box| superduper.components.model:672  | Adding 187 model outputs to `db`
2025-Jan-13 12:45:27.97| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:310  | Inserted 336 documents into _outputs__chunker_sample_text_vector_search__fc9ea0798a1e47e3
2025-Jan-13 12:45:27.97| INFO     | Duncans-MBP.fritz.box| superduper.backends.local.queue:120  | Consumed all events


## Select outputs of upstream listener

:::note
This is useful if you have performed a first step, such as pre-computing 
features, or chunking your data. You can use this query to 
operate on those outputs.
:::

<!-- TABS -->
## Build text embedding model

OpenAI:

In [10]:
from superduper.components.datatype import Vector
from superduper_openai import OpenAIEmbedding

openai_embedding = OpenAIEmbedding(
    identifier='text-embedding-ada-002',
    datatype=Vector(shape=(1536,)),
)

Sentence-transformers

In [11]:
from superduper_sentence_transformers import SentenceTransformer

sentence_transformers_embedding = SentenceTransformer(
    identifier="sentence-transformers-embedding",
    model="BAAI/bge-small-en",
    datatype=Vector(shape=(1024,)),
    postprocess=lambda x: x.numpy(),
    predict_kwargs={"show_progress_bar": True},
)

In [12]:
from superduper.components.model import ModelRouter

embedding_model = ModelRouter(
    'embedding',
    models={'openai': openai_embedding, 'sentence_transformers': sentence_transformers_embedding},
    model='<var:embedding_model>' if not APPLY else 'openai',
    example='this is a test',
)

## Create vector-index

In [13]:
from superduper import VectorIndex, Listener

vector_index_name = f'vector-index-{COLLECTION_NAME}'

vector_index = VectorIndex(
    vector_index_name,
    indexing_listener=Listener(
        key=upstream_listener.outputs,
        select=db[upstream_listener.outputs].select(),
        model=embedding_model,
        identifier=f'embedding-listener-{COLLECTION_NAME}',
        upstream=[upstream_listener],
    )
)

In [14]:
if APPLY:
    db.apply(vector_index, force=True)

2025-Jan-13 12:45:38.58| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf ID already exists
2025-Jan-13 12:45:38.58| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
2025-Jan-13 12:45:40.20| WARNING  | Duncans-MBP.fritz.box| superduper.components.listener:76   | output_table not found in listener.dict()
2025-Jan-13 12:45:40.20| WARNING  | Duncans-MBP.fritz.box| superduper.components.listener:76   | output_table not found in listener.dict()
2025-Jan-13 12:45:40.21| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:265  | Found identical model:chunk_model:76436da6a29d4031
2025-Jan-13 12:45:40.21| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:341  | Found update model:chunk_model:76436da6a29d4031
2025-Jan-13 12:45:40.21| WARNING  | Duncans-MBP.fritz.box| superduper.backends.local.artifacts:87   | File /tmp/test_db/85d22fdc19a5621624255e5a6a0c9427d82370fd already exists
2025-Jan-13 12:45:40.21| WARNING  | Duncan

vector-index-sample_text_vector_search
└── chunk_model
    ├── status: update
    ├── changes
    │   └── _object: &:blob:85d22fdc19a5621624255e5a6a0c9427d82370fd
    └── type_id: model

2025-Jan-13 12:45:43.40| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:79   | Found these changes and/ or additions that need to be made:
2025-Jan-13 12:45:43.40| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:81   | ----------------------------------------------------------------------------------------------------
2025-Jan-13 12:45:43.40| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:82   | METADATA EVENTS:
2025-Jan-13 12:45:43.40| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:83   | ----------------------------------------------------------------------------------------------------
2025-Jan-13 12:45:43.40| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:92   | [0]: model:chunk_model:76436da6a29d4031: update
2025-Jan-13 12:45:43.40| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:90   | [1]: datatype:vector[1536]:e9e26ef6bb634368: create ~ [6]
2025-Jan-13 12:45:43.40| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:90   |

336it [00:00, 17893.20it/s]


2025-Jan-13 12:45:45.63| INFO     | Duncans-MBP.fritz.box| superduper.components.model:1345 | Predicting with model openai


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.66s/it]


2025-Jan-13 12:45:52.25| INFO     | Duncans-MBP.fritz.box| superduper.components.model:672  | Adding 336 model outputs to `db`
2025-Jan-13 12:45:53.85| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:310  | Inserted 336 documents into _outputs__embedding-listener-sample_text_vector_search__13c7d67953a04170
2025-Jan-13 12:45:54.01| INFO     | Duncans-MBP.fritz.box| superduper.backends.local.queue:120  | Consumed all events


By applying the RAG model to the database, it will subsequently be accessible for use in other services.

In [15]:
from superduper import Application

app = Application(
    f'text-vector-search-app-{COLLECTION_NAME}',
    components=[
        upstream_listener,
        vector_index,
    ]
)

2025-Jan-13 12:45:56.04| INFO     | Duncans-MBP.fritz.box| superduper.components.application:39   | Resorting components based on topological order.
2025-Jan-13 12:45:56.04| INFO     | Duncans-MBP.fritz.box| superduper.components.application:56   | New order of components: ['listener:chunker_sample_text_vector_search:fc9ea0798a1e47e3', 'vector_index:vector-index-sample_text_vector_search:3a06ad54829f4776']


In [16]:
if APPLY:
    db.apply(app, force=True)

2025-Jan-13 12:45:58.23| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf ID already exists
2025-Jan-13 12:45:58.23| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
2025-Jan-13 12:45:58.24| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf ID already exists
2025-Jan-13 12:45:58.24| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
2025-Jan-13 12:45:59.86| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf ID already exists
2025-Jan-13 12:45:59.86| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
2025-Jan-13 12:45:59.86| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf ID already exists
2025-Jan-13 12:45:59.86| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
2025-Jan-13 12:46:01.49| WARNING  | Duncans-MBP.fritz.box| superduper.components.lis

text-vector-search-app-sample_text_vector_search
├── chunk_model
│   ├── status: update
│   ├── changes
│   │   └── _object: &:blob:85d22fdc19a5621624255e5a6a0c9427d82370fd
│   └── type_id: model
├── vector[1536]
│   ├── status: update
│   ├── changes
│   │   └── shape: (1536,)
│   └── type_id: datatype
├── vector[1024]
│   ├── status: update
│   ├── changes
│   │   └── shape: (1024,)
│   └── type_id: datatype
├── sentence-transformers-embedding
│   ├── status: update
│   ├── changes
│   │   ├── preferred_devices: ('cuda', 'mps', 'cpu')
│   │   └── postprocess: &:blob:7e3797ba5e8838fba8c62d6fe57f2cfe04a96e95
│   └── type_id: model
└── embedding
    ├── status: update
    ├── changes
    │   └── signature: *args,**kwargs
    └── type_id: model

2025-Jan-13 12:46:06.35| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:79   | Found these changes and/ or additions that need to be made:
2025-Jan-13 12:46:06.35| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:81   | ----------------------------------------------------------------------------------------------------
2025-Jan-13 12:46:06.35| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:82   | METADATA EVENTS:
2025-Jan-13 12:46:06.35| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:83   | ----------------------------------------------------------------------------------------------------
2025-Jan-13 12:46:06.35| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:92   | [0]: model:chunk_model:76436da6a29d4031: update
2025-Jan-13 12:46:06.35| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:92   | [1]: datatype:vector[1536]:e9e26ef6bb634368: update
2025-Jan-13 12:46:06.35| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:90   | [2]: 

You can now load the model elsewhere and make predictions using the following command.

In [17]:
search_term = 'tell me about the use of pylance and vector-search'

vector_search_query = db[f'_outputs__chunker_{COLLECTION_NAME}'].like(
    {f'_outputs__chunker_{COLLECTION_NAME}': search_term},
    n=10,
    vector_index=vector_index_name,
).select()

In [19]:
if APPLY:
    print(vector_search_query.tolist())

2025-Jan-13 12:46:23.20| WARNING  | Duncans-MBP.fritz.box| superduper.backends.base.query:497  | Error in completing UUIDs from cache: 'chunker_sample_text_vector_search'
2025-Jan-13 12:46:23.20| WARNING  | Duncans-MBP.fritz.box| superduper.backends.base.query:497  | Error in completing UUIDs from cache: 'chunker_sample_text_vector_search'
2025-Jan-13 12:46:23.21| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:802  | Getting vector-index
2025-Jan-13 12:46:23.21| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:810  | {}
2025-Jan-13 12:46:23.21| INFO     | Duncans-MBP.fritz.box| superduper.components.model:1335 | Predicting with model openai
[Document({'_outputs__chunker_sample_text_vector_search__fc9ea0798a1e47e3': "--- sidebar_position: 7 --- # Vector-search Superduper allows users to implement vector-search in their database by either using in-database functionality, or via a sidecar implementation with `lance` and `FastAPI`. ## Philosophy In Superduper, from 

In [21]:
from superduper import QueryTemplate, CFG

qt = QueryTemplate(
    'vector_search',
    template=vector_search_query,
    substitutions={
        COLLECTION_NAME: 'table_name',
        search_term: 'search_term',
        'mongodb': 'data_backend',
    },
    types={
        'search_term': {
            'type': 'str',
            'default': 'enter your question here...',
        },
        'table_name': {
            'type': 'str',
            'default': 'sample_text_vector_search'
        },
        'data_backend': {
            'type': 'mongodb',
            'choices': ['mongodb', 'ibis'],
            'default': 'mongodb'
        }
    },
    db=db
)

## Create template

In [23]:
from superduper import Template, CFG, Table, Schema
from superduper.components.dataset import RemoteData

template = Template(
    'text_vector_search',
    template=app,
    default_tables=[Table(
        'sample_text_vector_search',
        schema=Schema('sample_text_vector_search/schema', fields={'x': 'str'}),
        data=RemoteData(
            'superduper-docs',
            getter=getter,
        )
    )],
    queries=[qt],
    substitutions={COLLECTION_NAME: 'table_name', 'mongodb': 'data_backend'},
    template_variables=['embedding_model', 'table_name', 'data_backend'],
    types={
        'embedding_model': {
            'type': 'str',
            'choices': ['openai', 'sentence_transformers'],
            'default': 'openai',
        },
        'table_name': {
            'type': 'str',
            'default': 'sample_text_vector_search'
        },
        'data_backend': {
            'type': 'mongodb',
            'choices': ['mongodb', 'ibis'],
            'default': 'mongodb'
        }
    },
    db=db
)

2025-Jan-13 12:46:50.73| WARNING  | Duncans-MBP.fritz.box| superduper.components.listener:76   | output_table not found in listener.dict()
2025-Jan-13 12:46:50.74| WARNING  | Duncans-MBP.fritz.box| superduper.components.listener:76   | output_table not found in listener.dict()
2025-Jan-13 12:46:50.74| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf listener:chunker_sample_text_vector_search already exists
2025-Jan-13 12:46:50.74| WARNING  | Duncans-MBP.fritz.box| superduper.components.listener:76   | output_table not found in listener.dict()
2025-Jan-13 12:46:50.74| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf model:chunk_model already exists
2025-Jan-13 12:46:50.75| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
2025-Jan-13 12:46:50.75| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf sample-text-vector-search-select already exists
2025-Jan-13 12:46:50.75| WARNING  | Dunca

In [24]:
template.export('.')

2025-Jan-13 12:46:52.96| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:564  | Leaf str already exists
